2- Escolha um conjunto de textos  (poemas, letras de música, etc )

a) Separe 10% dos textos como conjunto de testes e e o restante como conjunto de treinamento

b) Construa um modelo de linguagen  explorando diferentes n-gramas (unigrama, bigrama e trigrama) e métodos de suavização (Laplace vs Lidstone)

c) Faça uma análise do melhor modelo de linguagem com base na sua perplexidade no conjunto de teste

d) Escolha o melhor modelo e gere um texto de até 30 tokens predizendo a proxima palavra de maxima probabilidade.

In [1]:
import nltk
from nltk import bigrams, trigrams
from nltk.lm import MLE, Laplace, Lidstone
from nltk.lm.preprocessing import pad_both_ends
import re
import random
import string
from nltk.lm.preprocessing import padded_everygram_pipeline
nltk.download('stopwords')
nltk.download('punkt')

def pre_processamento(texto):

    # seleciona apenas letras e coloca todas em minúsculo
    letras_min =  re.findall(r'\b[A-zÀ-úü]+\b', texto.lower())

    # remove stopwords
    stopwords = nltk.corpus.stopwords.words('portuguese')
    stop = set(stopwords)
    sem_stopwords = [w for w in letras_min if w not in string.punctuation]


    # juntando os tokens novamente em formato de texto
    texto_limpo = " ".join(sem_stopwords)

    return texto_limpo

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# id do corpus
# no nosso caso estamos usando id machado
nltk_id = 'machado'

# eh necessario baixar o corpus
nltk.download(nltk_id)

# agora o corpus esta acessivel
# visualizando as obras disponiveis

#print(nltk.corpus.machado.readme())
# ou

#print(nltk.corpus.machado.fileids())

# apos escolher a obra
# salvamos a string em uma variavel
ressurreicao = pre_processamento(nltk.corpus.machado.raw('romance/marm01.txt'))
mao_luva = pre_processamento(nltk.corpus.machado.raw('romance/marm02.txt'))
helena = pre_processamento(nltk.corpus.machado.raw('romance/marm03.txt'))
iaiá_garcia = pre_processamento(nltk.corpus.machado.raw('romance/marm04.txt'))
memorias_postumas= pre_processamento(nltk.corpus.machado.raw('romance/marm05.txt'))
casa_velha = pre_processamento(nltk.corpus.machado.raw('romance/marm06.txt'))
quincas_borba = pre_processamento(nltk.corpus.machado.raw('romance/marm07.txt'))
dom_casmurro = pre_processamento(nltk.corpus.machado.raw('romance/marm08.txt'))
esau_jaco = pre_processamento(nltk.corpus.machado.raw('romance/marm09.txt'))
memorial_aires = pre_processamento(nltk.corpus.machado.raw('romance/marm10.txt'))



[nltk_data] Downloading package machado to /root/nltk_data...
[nltk_data]   Package machado is already up-to-date!


In [3]:
textos = [ressurreicao, mao_luva, helena,iaiá_garcia, memorias_postumas, casa_velha,
          quincas_borba, dom_casmurro, esau_jaco, memorial_aires]

tamanho_treinamento = int(len(textos) * 0.9)
tamanho_teste = len(textos) - tamanho_treinamento

random.shuffle(textos)

conjunto_treinamento = textos[:tamanho_treinamento]
conjunto_teste = textos[tamanho_treinamento:]
"""
print("Conjunto de Treinamento:")
for texto in conjunto_treinamento:
    print(texto)

print("\nConjunto de Teste:")
for texto in conjunto_teste:
    print(texto)"""

'\nprint("Conjunto de Treinamento:")\nfor texto in conjunto_treinamento:\n    print(texto)\n\nprint("\nConjunto de Teste:")\nfor texto in conjunto_teste:\n    print(texto)'

```
# 1. paddedLine = [list(pad_both_ends(nltk.word_tokenize(texto), n=2))]
# 2. train, vocab = padded_everygram_pipeline(2, paddedLine)
# 3. lmMLE = MLE(2)
# 4. lmLAP = Laplace(2)
# 5. lmLIDS = Lidstone(0.1, 2)
# 6. lmMLE.fit(train, vocabulary_text=vocab)
# 7. lmLAP.fit(train, vocabulary_text=vocab)
# 8. lmLIDS.fit(train, vocabulary_text=vocab)
```
 A linha 1 em questão tokeniza o texto em palavras, adiciona tokens < s> e < /s> de início e fim, criando bigramas apropriados, e coloca o resultado em uma lista.


 A linha 2 em questão cria o conjunto de treinamento e o vocabulário necessários para treinar o modelo de linguagem com bigramas (n-gramas de tamanho 2)


 Nas linhas 3,4,5,6 é realizado em sequência a criação dos modelos de linguagens(MLE) de bigramas, modelo de linguagem com suavização Laplace(Laplace(2)) de bigrama e o modelo de linguagem com suavização Lidstone(Lidstone(0.1,2)) (Todos são bigramas)


Na linha seguinte é feito o treinamento dos modelos.

In [26]:
# Exemplo de texto
texto = pre_processamento(nltk.corpus.machado.raw('romance/marm01.txt'))

indice_90_percent = int(0.9 * len(texto))

texto_90_percent = nltk.word_tokenize(texto[:indice_90_percent])
texto_10_percent = nltk.word_tokenize(texto[indice_90_percent:])


paddedLine = [list(pad_both_ends(texto_90_percent, n=2))]


trainMLE, vocabMLE = padded_everygram_pipeline(3, paddedLine)
trainLAP, vocabLAP = padded_everygram_pipeline(3, paddedLine)
trainLIDS, vocabLIDS = padded_everygram_pipeline(3, paddedLine)

lmMLE = MLE(3)
lmLAP = Laplace(3)
lmLIDS = Lidstone(0.1,3)

lmLIDS.fit(trainLIDS, vocabulary_text=vocabLIDS)
lmLAP.fit(trainLAP, vocabulary_text=vocabLAP)
lmMLE.fit(trainMLE, vocabulary_text=vocabMLE)




In [34]:
trigramas_test = [list(trigrams(pad_both_ends(texto_10_percent, n=2)))]

In [44]:
print(lmLIDS.perplexity(trigramas_test))
print(lmLIDS.generate(30))

5561.000000000008
['félix', 'tanto', 'mais', 'disse', 'ele', 'há', 'muito', 'tempo', 'mais', 'meteu', 'se', 'nele', 'e', 'mandou', 'chamar', 'o', 'médico', 'e', 'dirigiu', 'se', 'para', 'o', 'médico', 'era', 'impossível', 'entretanto', 'não', 'era', 'assim', 'aparentemente']


In [36]:
print(lmLAP.perplexity(trigramas_test))
print(lmLAP.generate(30))

5561.000000000008
['da', 'janela', 'com', 'o', 'tempo', 'há', 'amigos', 'de', 'meu', 'irmão', 'falava', 'dele', 'com', 'os', 'derradeiros', 'lampejos', 'do', 'crepúsculo', 'até', 'que', 'perderia', 'eu', 'com', 'isso', 'a', 'que', 'ela', 'pretendia', 'fazer', 'caminhavam']


In [37]:
print(lmMLE.perplexity(trigramas_test))
print(lmMLE.generate(30))

inf
['tínhamos', 'veio', 'tirar', 'no', 'la', 'o', 'assunto', 'no', 'chão', 'posso', 'ir', 'à', 'chácara', 'podes', 'leva', 'o', 'clara', 'luís', 'deitou', 'a', 'correr', 'as', 'lágrimas', 'choradas', 'durante', 'aqueles', 'oito', 'dias', 'e', 'indiferentes']
